**В этом ноутбуке показаны попытки 1-1.3 создания рек.системы**  
Процесс создания признаков для трен/тест датасетов показан в   
главном ноутбуке с лучшей попыткой 6.  
Ниже краткое описание различий каждой попытки

**Ver_1**  
- 50 кандидатов на сессию
- фичи : \
Aid \
'item_quantity_in_df' Кол-во aid в датасете.   
'sessions_w_item' Сколько session с этим aid. \
'mean_item_type' Средний тип этого aid. \
Session \
'session_quantity_in_df' Кол-во этой session в датасете \
'items_in_session' Сколько aid в этой session \
'mean_session_type' Какой средний тип aid в этой session \
- 1/3 train data


**Ver_1.1**  
- Все как в  Ver_1, кроме undersampling
- 0.1 ratio - clicks, 0,01 ratio - carts, orders


**Ver_1.2**  
- Все как в  Ver_1, кроме undersampling
- 0.3 ratio - clicks, 0.05 ratio - carts, orders

**Ver_1.3**  
- Все как в  Ver_1, кроме undersampling
- 0.4 ratio - clicks, 0.2 ratio - carts, 0.2 ratio - orders

In [ ]:
# main_path = '/Users/stanislavkrupnov/Jup.Notebook'
main_path = '/content/drive/Othercomputers/Mac/Jup.Notebook'

# Функции

In [ ]:
def metric_eval(predictions: pd.DataFrame, valid: pd.DataFrame,
                id2type: dict) -> tuple:
    """
    Вычисляет метрики для оценки качества модели.

    Parameters:
    - predictions (pd.DataFrame): DataFrame с предсказаниями модели.
    - valid (pd.DataFrame): DataFrame с данными для валидации.
    - id2type (dict): Словарь для преобразования индексов типов в соответствующие строки.

    Returns:
    - tuple: Кортеж, содержащий локальную метрику и реколл для каждого типа.
    """
    # Преобразование индексов типов в соответствующие строки
    valid.type = valid.type.map(lambda idx: id2type[idx])

    # Создание списка с метками для каждой сессии и типа
    ground_truth = valid.groupby(['session', 'type'])['aid'].apply(list)
    ground_truth = ground_truth.reset_index().rename(columns={'aid': 'labels'})

    # Обрезка меток для типа 'clicks' до одной метки
    ground_truth.loc[ground_truth.type == 'clicks',
                     'labels'] = ground_truth.loc[ground_truth.type ==
                                                  'clicks', 'labels'].str[:1]

    # Объединение предсказаний с истинными метками
    submission_with_gt = predictions.merge(
        ground_truth[['session', 'type', 'labels']],
        how='left',
        on=['session', 'type'])

    # Отбрасывание сессий без истинных меток
    submission_with_gt = submission_with_gt[~submission_with_gt.labels_y.isna(
    )]

    # Вычисление количества совпадений между предсказанными и истинными метками
    submission_with_gt['hits'] = submission_with_gt.apply(
        lambda df: len(set(df.labels_x).intersection(set(df.labels_y))),
        axis=1)

    # Вычисление количества истинных меток для каждого типа
    submission_with_gt['gt_count'] = submission_with_gt.labels_y.str.len(
    ).clip(0, 20)

    # Вычисление реколла для каждого типа
    recall_per_type = submission_with_gt.groupby([
        'type'
    ])['hits'].sum() / submission_with_gt.groupby(['type'])['gt_count'].sum()

    # Вычисление локальной метрики на основе реколла для каждого типа
    local_validation_score = (recall_per_type * pd.Series({
        'clicks': 0.10,
        'carts': 0.30,
        'orders': 0.60
    })).sum()

    return local_validation_score, recall_per_type

In [ ]:
def read_covis_to_dict(cov_ver: str, cov_type: str, n: int) -> dict:
    '''
    Читает ковиз-матрицу из нескольких файлов и возвращает словарь.

    Parameters:
    - cov_ver (str): Версия ковиз-матрицы.
    - cov_type (str): Тип ковиз-матрицы (b_tr или lv).
    - n (int): Количество файлов с матрицей.

    Returns:
    - dict: Словарь, представляющий ковиз-матрицу.
    '''
    for k in tqdm(range(0, n)):
        path = f'{main_path}/c/{cov_type}_{cov_ver}/{cov_type}_covis_matrix_{k}.parquet'

        if k == 0:
            covis_matrix = pd.read_parquet(path)
            covis_matrix = covis_matrix.groupby('aid_x').aid_y.apply(
                list).to_dict()
        else:
            df = pd.read_parquet(path)
            df = df.groupby('aid_x').aid_y.apply(list).to_dict()
            covis_matrix.update(df)

    return covis_matrix

In [ ]:
def read_parquet_in_chunks(file_path: str, batch_num: int):
    '''
    Читает Parquet-файл порциями и возвращает DataFrame.

    Parameters:
    - file_path (str): Путь к Parquet-файлу.
    - batch_num (int): Количество порций для чтения.

    Yields:
    - pd.DataFrame: DataFrame, представляющий порцию данных из Parquet-файла.
    '''
    table = pq.read_table(file_path)
    num_rows = len(table)
    batch_size = num_rows // batch_num

    for i in range(0, num_rows, batch_size):
        yield table[i:i + batch_size].to_pandas()

In [ ]:
def create_labels(n: int,
                  inp_dict: dict,
                  covis: dict,
                  top_pop_aids: set,
                  cov_f: bool,
                  b: int = -1) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений.

    Parameters:
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.
    - b (int): Номер файла для сохранения, если cov_f=True.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    featuress = {}

    for session, aids in tqdm(inp_dict.items()):
        # Инициализация словаря для подсчета aids
        aid_counter = Counter()

        # Цикл по всем aids в сессии
        for aid in aids:
            # Если aid есть в covis_matrix, увеличиваем его счетчик
            if aid in covis:
                aid_counter.update(covis[aid])

        if len(set(aids)) >= n:  # Оставлю только последние n уникальные aids
            labels[session] = list(dict.fromkeys(aids))[-n:]
        else:
            # Получаем топ встречающихся aids
            top_aids = [
                aid
                for aid, count in aid_counter.most_common(n - len(set(aids)))
            ]

            # Добавляем исходные aids и топ встречающихся aids в labels
            labels[session] = list(set(list(set(aids)) + list(set(top_aids))))

            if len(set(aids)) < n:
                pops = list(top_pop_aids.difference(set(labels[session])))
                labels[session] += pops[:n - len(labels[session])]

        if cov_f == False:
            continue
        else:
            # Создание словаря с признаком - рангом (по ко-виз матрице) каждого кандидата внутри сессии
            for aid in labels[session]:
                if aid in list(aid_counter.keys()):
                    if session not in featuress:
                        featuress[session] = {}
                    featuress[session][aid] = aid_counter[aid]

    if cov_f == True:
        if b == -1:  # Если мы в функции create_labels
            path_ = f'{main_path}/tr/{ver_folder}/featuress.pickle'
        else:  # Если мы в функции create_labels_in_batches
            path_ = f'{main_path}/tr/{ver_folder}/featuress_{b}.pickle'

        with open(path_, 'wb') as file:
            pickle.dump(featuress, file)

        print(
            f'Словарь с признаком-рангом по ковиз-матрице сохранен в : {path_}'
        )

    return labels

In [ ]:
def create_labels_in_batches(k: int, n: int, inp_dict: dict, covis: dict,
                             top_pop_aids: set, cov_f: bool) -> dict:
    '''
    Создает метки для сессий на основе матрицы ко-посещений по батчам.

    Parameters:
    - k (int): Количество батчей.
    - n (int): Количество кандидатов на сессию.
    - inp_dict (dict): Словарь с записями сессий.
    - covis (dict): Матрица ко-посещений.
    - top_pop_aids (set): Топ популярных товаров.
    - cov_f (bool): Создавать ли признак из ковиз-матрицы.

    Returns:
    - dict: Словарь с кандидатами.
    '''
    labels = {}
    partt = len(inp_dict) // k

    for batch in tqdm(range(k), desc='part'):

        start = partt * batch
        end = partt * (batch + 1)
        slicee_ = list(inp_dict.items())[start:end]
        slicee = dict(slicee_)

        labels_ = create_labels(n=n,
                                inp_dict=slicee,
                                covis=covis,
                                top_pop_aids=top_pop_aids,
                                cov_f=cov_f,
                                b=batch)
        labels.update(labels_)

    return labels

In [ ]:
def create_pop_features(for_features_df: pd.DataFrame) -> pd.DataFrame:
    '''
    Создает признак ранжирования aid по встречаемости в группах по типам.

    Parameters:
    - for_features_df (pd.DataFrame): Датасет с данными для признака.

    Returns:
    - pd.DataFrame: Датасет с признаком.
    '''
    # Создадим пустой DataFrame, в который будем записывать результаты
    result_df = pd.DataFrame()

    # Сгруппируем исходный DataFrame по столбцу 'type'
    grouped = for_features_df.groupby('type')

    id_2_type = {0: 'clicks', 1: 'carts', 2: 'orders'}

    # Пройдемся по каждой группе и выполним ранжирование aid по встречаемости
    for name, group in tqdm(grouped):
        # Считаем количество вхождений каждого aid в текущей группе и создадим временный DataFrame
        aid_counts = group['aid'].value_counts().reset_index()
        aid_counts.columns = ['aid', 'count']

        # Добавим столбец с рангом, используя функцию rank()
        aid_counts['rank'] = aid_counts['count'].rank(ascending=False,
                                                      method='dense')
        aid_counts.drop(columns=['count'], inplace=True)

        # Присоединим результат к исходной группе и добавим в общий результатный DataFrame
        merged_group = pd.merge(group, aid_counts, on='aid', how='left')

        # Создадим имена для столбцов рангов на основе типа 'type'
        rank_column_name = 'rank_' + id_2_type[name]
        merged_group.rename(columns={'rank': rank_column_name}, inplace=True)

        result_df = pd.concat([result_df, merged_group], ignore_index=True)

        result_df.fillna(-1, inplace=True)
        result_df.rank_clicks = result_df.rank_clicks.astype(np.int32)
        result_df.rank_carts = result_df.rank_carts.astype(np.int32)
        result_df.rank_orders = result_df.rank_orders.astype(np.int32)
        result_df.reset_index(inplace=True, drop=True)

    return result_df

In [ ]:
def add_targets(dataframe_: pd.DataFrame, targets: pd.DataFrame,
                targ_type: int) -> pd.DataFrame:
    '''
    Добавляет таргеты к датафрейму.

    Parameters:
    - dataframe_ (pd.DataFrame): Датафрейм, которому нужны таргеты.
    - targets (pd.DataFrame): Датафрейм с таргетами всех типов.
    - targ_type (int): Текущий тип таргета.

    Returns:
    - pd.DataFrame: Датафрейм с нужными таргетами.
    '''
    targets_ = targets.loc[targets['type'] == targ_type,
                           ['session', 'aid', 'type']].rename(columns={
                               'type': 'target'
                           }).copy()
    targets_['target'] = 1
    targets_ = targets_.drop_duplicates()
    merge_cols = ["session", "aid"]
    dataframe = dataframe_.merge(targets_, on=merge_cols, how="left")
    dataframe = dataframe.fillna(0)
    dataframe.target = dataframe.target.astype(np.uint8)

    return dataframe

In [ ]:
def w_pbar(pbar: tqdm, func: callable):
    '''
    Оборачивает функцию, обновляя tqdm прогресс бар.

    Parameters:
    - pbar (tqdm): Прогресс бар.
    - func (callable): Функция.

    Returns:
    - callable: Обернутая функция.
    '''
    def foo(*args, **kwargs):
        pbar.update(1)
        return func(*args, **kwargs)

    return foo

# import

In [ ]:
import os
import sys
from tqdm import tqdm
import glob
import pyarrow.parquet as pq
import pickle
import pandas as pd
import gc
import numpy as np
import itertools
from collections import Counter
import random
from itertools import product
import xgboost as xgb
from sklearn.model_selection import GroupKFold
import gdown
from google.colab import drive
drive.mount('/content/drive')
import polars as pl
import shutil
from datetime import datetime as dt
from collections import defaultdict


Mounted at /content/drive


# Ver_1

In [ ]:
ver = 1
ver_folder = f'ver_{ver}'
cand_n = 50

## train_candidates

### Создание списка tr_candidates

In [ ]:
tr_candidates_sessions = pd.read_parquet(f'{main_path}/s/tr_candidates_sessions.parquet')
tr_2_first_w = pd.read_parquet(f'{main_path}/s/tr_2_first_w.parquet')

In [ ]:
top_pop_aids = pd.concat([tr_candidates_sessions, tr_2_first_w])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'b_tr',
                                  n = 20)

100%|███████████████████████████████████████████| 20/20 [00:15<00:00,  1.26it/s]


In [ ]:
%%time
# Преобразование датафрейма в словарь
tgl_dict = tr_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

CPU times: user 29.4 s, sys: 1.27 s, total: 30.6 s
Wall time: 30.7 s


In [ ]:

len(tgl_dict)//3

1547581

In [ ]:
# Если нужен слайс
# last_ses_in_1div100_part =
slicee_ = list(tgl_dict.items())[:1547581] # 1547581 - одна третья данных
slicee = dict(slicee_)
tgl_dict = slicee

In [ ]:
with open(f'{main_path}/tgl_1div3_data_sessions.pickle', 'wb') as file:
    pickle.dump(list(tgl_dict.keys()), file)

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = tgl_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = False)

100%|██████████████████████████████| 1547581/1547581 [01:01<00:00, 25235.69it/s]


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|████████████████████████████| 1547581/1547581 [00:00<00:00, 6977908.76it/s]


In [ ]:
tr_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
tr_candidates_['session'] = tr_candidates_['session'].astype(np.int32)

tr_candidates_ = tr_candidates_.explode('labels')

tr_candidates_['labels'] = tr_candidates_['labels'].astype(np.int32)

tr_candidates_.to_parquet(f'{main_path}/tr/{ver_folder}/tr_candidates.parquet')

### Добавление признаков, таргетов к tr_candidates

In [ ]:
tr_item_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_item_features.parquet')
tr_user_features = pd.read_parquet(f'{main_path}/tr/{ver_folder}/tr_user_features.parquet')

In [ ]:
targets = pd.read_parquet(f'{main_path}/s/targets.parquet')

**Добавление признаков + таргетов**

In [ ]:
tr_candidates = pd.DataFrame()
x = 0

for chunk in tqdm(read_parquet_in_chunks(file_path = f'{main_path}/tr/{ver_folder}/tr_candidates.parquet',
                       batch_num = 10)):

    chunk = chunk.rename(columns = {'labels': 'aid'})
    chunk = chunk.merge(tr_item_features, on = 'aid', how='left').fillna(-1)
    chunk = chunk.merge(tr_user_features, on = 'session', how='left').fillna(-1)

#     tr_candidates = pd.concat([tr_candidates, chunk])

    for index, type_ in tqdm(enumerate(['clicks', 'carts', 'orders'])):

        tr_candidates_ = add_targets(dataframe_ = chunk,
                                     targets = targets,
                                     targ_type = index)

        folder_path = f'{main_path}/tr/{ver_folder}/tr_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        tr_candidates_.to_parquet(f"{folder_path}/tr_candidates_{type_}_{x}.parquet")

    x += 1

0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:08,  8.80s/it]
2it [00:11,  5.05s/it]
3it [00:13,  4.48s/it]
1it [00:15, 15.78s/it]
0it [00:00, ?it/s]
1it [00:07,  7.62s/it]
2it [00:10,  4.54s/it]
3it [00:12,  4.03s/it]
2it [00:29, 14.71s/it]
0it [00:00, ?it/s]
1it [00:07,  7.78s/it]
2it [00:10,  4.56s/it]
3it [00:12,  4.05s/it]
3it [00:43, 14.36s/it]
0it [00:00, ?it/s]
1it [00:07,  7.93s/it]
2it [00:10,  4.67s/it]
3it [00:12,  4.15s/it]
4it [00:57, 14.32s/it]
0it [00:00, ?it/s]
1it [00:07,  7.82s/it]
2it [00:10,  4.59s/it]
3it [00:12,  4.07s/it]
5it [01:11, 14.22s/it]
0it [00:00, ?it/s]
1it [00:07,  7.77s/it]
2it [00:10,  4.64s/it]
3it [00:12,  4.09s/it]
6it [01:26, 14.18s/it]
0it [00:00, ?it/s]
1it [00:07,  7.68s/it]
2it [00:10,  4.53s/it]
3it [00:12,  4.03s/it]
7it [01:40, 14.10s/it]
0it [00:00, ?it/s]
1it [00:07,  7.72s/it]
2it [00:10,  4.54s/it]
3it [00:12,  4.02s/it]
8it [01:53, 14.04s/it]
0it [00:00, ?it/s]
1it [00:07,  7.64s/it]
2it [00:09,  4.48s/it]
3it [00:11,  3.99s/it]
9it [

In [ ]:
x = tr_candidates_.groupby('session')['aid'].count().reset_index()

In [ ]:
xx = tr_candidates_.groupby('session')['aid'].nunique().reset_index()

In [ ]:
a = 0
for session in tqdm(x['session']) :

    if (x.loc[x['session'] == session, 'aid'] != xx.loc[xx['session'] == session, 'aid']).any():
        print(session)
        a+=1

## test_candidates

### Создание списка test_candidates CPU ONLY

In [ ]:
test_candidates_sessions = pd.read_parquet(f'{main_path}/s/test_candidates_sessions.parquet')
tr_3_first_w = pd.read_parquet(f'{main_path}/s/tr_3_first_w.parquet')

In [ ]:
top_pop_aids = pd.concat([tr_3_first_w, test_candidates_sessions])
top_pop_aids = set(top_pop_aids['aid'].value_counts().index[:cand_n].tolist())

In [ ]:
covis_matrix = read_covis_to_dict(cov_ver = 'base',
                                  cov_type = 'lv',
                                  n = 80)

100%|██████████| 80/80 [02:22<00:00,  1.78s/it]


In [ ]:
# Преобразование датафрейма в словарь
test_dict = test_candidates_sessions.sort_values(["session", "ts"]).groupby('session')['aid'].apply(list).to_dict()

In [ ]:
sizee = len(test_dict) // 3

In [ ]:
# Если нужен слайс
slicee_ = list(test_dict.items())[:sizee] # sizee - одна третья данных
slicee = dict(slicee_)
test_dict = slicee

In [ ]:
with open(f'{main_path}/lv/test_1div3_data_sessions.pickle', 'wb') as file:
    pickle.dump(list(test_dict.keys()), file)

In [ ]:
labels = create_labels(n = cand_n,
                      inp_dict = test_dict,
                      covis = covis_matrix,
                      top_pop_aids = top_pop_aids,
                      cov_f = False)

100%|██████████| 600417/600417 [00:51<00:00, 11568.78it/s]


In [ ]:
sessions = []
aid_lists = []

for session, aids in tqdm(labels.items()):
    sessions.append(session)
    aid_lists.append(aids)

100%|██████████| 600417/600417 [00:00<00:00, 2135554.62it/s]


In [ ]:
test_candidates_ = pd.DataFrame({'session': sessions, 'labels': aid_lists})
test_candidates_['session'] = test_candidates_['session'].astype(np.int32)

test_candidates_ = test_candidates_.explode('labels')

test_candidates_['labels'] = test_candidates_['labels'].astype(np.int32)

test_candidates_.to_parquet(f'{main_path}/lv/{ver_folder}/test_candidates.parquet')

In [ ]:
del test_candidates_

### Добавление признаков, таргетов к test_candidates

In [ ]:
lv_item_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_item_features.parquet')
lv_user_features = pd.read_parquet(f'{main_path}/lv/{ver_folder}/lv_user_features.parquet')

**Добавление признаков + таргетов**

In [ ]:
x = 0

for chunk in tqdm(read_parquet_in_chunks(file_path = f'{main_path}/lv/{ver_folder}/test_candidates.parquet',
                       batch_num = 10)):

    chunk = chunk.rename(columns = {'labels': 'aid'})
    chunk = chunk.merge(lv_item_features, on = 'aid', how='left').fillna(-1)
    chunk = chunk.merge(lv_user_features, on = 'session', how='left').fillna(-1)

    for type_ in tqdm(['clicks', 'carts', 'orders']):

        folder_path = f'{main_path}/lv/{ver_folder}/test_candidates_{type_}'
        os.makedirs(folder_path, exist_ok=True)

        chunk.to_parquet(f"{folder_path}/test_candidates_{type_}_{x}.parquet")

    x += 1

0it [00:00, ?it/s]
100%|██████████| 3/3 [00:01<00:00,  1.57it/s]
1it [00:06,  6.94s/it]
100%|██████████| 3/3 [00:01<00:00,  1.60it/s]
2it [00:13,  6.63s/it]
100%|██████████| 3/3 [00:02<00:00,  1.43it/s]
3it [00:20,  6.65s/it]
100%|██████████| 3/3 [00:02<00:00,  1.41it/s]
4it [00:26,  6.73s/it]
100%|██████████| 3/3 [00:01<00:00,  1.52it/s]
5it [00:33,  6.64s/it]
100%|██████████| 3/3 [00:02<00:00,  1.41it/s]
6it [00:40,  6.77s/it]
100%|██████████| 3/3 [00:02<00:00,  1.34it/s]
7it [00:47,  6.79s/it]
100%|██████████| 3/3 [00:01<00:00,  1.59it/s]
8it [00:53,  6.66s/it]
100%|██████████| 3/3 [00:02<00:00,  1.40it/s]
9it [01:00,  6.74s/it]
100%|██████████| 3/3 [00:02<00:00,  1.42it/s]
10it [01:07,  6.71s/it]


## Обучение модели

In [ ]:
ver = '1'
ver_folder = f'ver_{ver}'
typee = 'clicks'
# typee = 'carts'
# typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_folder}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[2 : -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.92707	valid-map:0.92766
[100]	train-map:0.92924	valid-map:0.92925
[200]	train-map:0.92991	valid-map:0.92951
[300]	train-map:0.93061	valid-map:0.92969
[400]	train-map:0.93106	valid-map:0.92972
[500]	train-map:0.93146	valid-map:0.92978
[600]	train-map:0.93189	valid-map:0.92983
[700]	train-map:0.93223	valid-map:0.92989
[800]	train-map:0.93259	valid-map:0.92988
[900]	train-map:0.93291	valid-map:0.92985
[999]	train-map:0.93317	valid-map:0.92984


1it [02:38, 158.97s/it]

[0]	train-map:0.92738	valid-map:0.92681
[100]	train-map:0.92953	valid-map:0.92831
[200]	train-map:0.93032	valid-map:0.92860
[300]	train-map:0.93094	valid-map:0.92873
[400]	train-map:0.93143	valid-map:0.92877
[500]	train-map:0.93192	valid-map:0.92878
[600]	train-map:0.93227	valid-map:0.92880
[700]	train-map:0.93259	valid-map:0.92877
[800]	train-map:0.93294	valid-map:0.92876
[900]	train-map:0.93327	valid-map:0.92877
[999]	train-map:0.93356	valid-map:0.92876


2it [05:08, 153.56s/it]

...

3it [07:38, 151.87s/it]

[0]	train-map:0.92739	valid-map:0.92661
[100]	train-map:0.92949	valid-map:0.92821
[200]	train-map:0.93031	valid-map:0.92840
[300]	train-map:0.93093	valid-map:0.92846
[400]	train-map:0.93140	valid-map:0.92852
[500]	train-map:0.93183	valid-map:0.92851
[600]	train-map:0.93223	valid-map:0.92856
[700]	train-map:0.93257	valid-map:0.92856
[800]	train-map:0.93289	valid-map:0.92854
[900]	train-map:0.93319	valid-map:0.92857
[999]	train-map:0.93347	valid-map:0.92855


4it [10:09, 151.44s/it]

[0]	train-map:0.92716	valid-map:0.92773
[100]	train-map:0.92934	valid-map:0.92927
[200]	train-map:0.93012	valid-map:0.92954
[300]	train-map:0.93070	valid-map:0.92962
[400]	train-map:0.93120	valid-map:0.92967
[500]	train-map:0.93159	valid-map:0.92966
[600]	train-map:0.93195	valid-map:0.92968
[700]	train-map:0.93231	valid-map:0.92966
[800]	train-map:0.93261	valid-map:0.92956
[900]	train-map:0.93293	valid-map:0.92961
[999]	train-map:0.93320	valid-map:0.92957


5it [12:39, 151.87s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_1/XGB_fold2_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1/XGB_fold4_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1/XGB_fold1_orders.xgb']

## Предсказание

In [ ]:
model_version = 1
ver_folder = f'ver_{model_version}'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_1/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_folder}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 10/10 [00:03<00:00,  2.78it/s]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020850, 8)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:29<00:00,  5.92s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_1/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
ver_folder = 'ver_1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')


In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{ver_folder}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.77it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.33303322356139226, type
carts     0.219707
clicks    0.233654
orders    0.406259
dtype: float64)


👁️**Model score : 0.333033**👁️

# Ver_1.1

Те же кандидаты (с версии 1) но именно предсказание другое, так как сохраненные настройки модели другие ( обучение с андерсемплингом)

## Обучение

In [ ]:
ver = '1.1'
ver_folder = f'ver_{ver}'
ver_prev = 'ver_1'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_prev}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 10/10 [00:22<00:00,  2.25s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

In [ ]:
#orders
tr_candidates['target'].value_counts()

0    77144955
1      234095
Name: target, dtype: int64

In [ ]:
#carts
tr_candidates['target'].value_counts()

0    77076368
1      302682
Name: target, dtype: int64

In [ ]:
#clicks
tr_candidates['target'].value_counts()

0    75316137
1     2062913
Name: target, dtype: int64

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.027390053209978095

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0039270402570084776

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0030344822937546597

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.1)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.01)  # Указываем желаемое соотношение классов

# # orders
under_sampler = RandomUnderSampler(sampling_strategy= 0.01)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    # Применяем under-sampling к данным
    # X_train, y_train = under_sampler.fit_resample(X_train, y_train)
    # X_valid, y_valid = under_sampler.fit_resample(X_valid, y_valid)

    # После under-sampling, получаем список с количеством объектов в каждой групп
    # X_for = tr_candidates.loc[train_idx, tr_candidates.columns[: -1]]
    # y_for = tr_candidates.loc[train_idx, 'target']
    # X_for, y_for = under_sampler.fit_resample(X_for, y_for)
    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    # dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) )
    # dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.94329	valid-map:0.94337
[100]	train-map:0.94610	valid-map:0.94517
[200]	train-map:0.94736	valid-map:0.94554
[300]	train-map:0.94826	valid-map:0.94576
[400]	train-map:0.94902	valid-map:0.94574
[500]	train-map:0.94973	valid-map:0.94579
[600]	train-map:0.95039	valid-map:0.94588
[700]	train-map:0.95095	valid-map:0.94584
[800]	train-map:0.95149	valid-map:0.94583
[900]	train-map:0.95201	valid-map:0.94587
[999]	train-map:0.95256	valid-map:0.94583


1it [00:59, 59.76s/it]

[0]	train-map:0.94341	valid-map:0.94292
[100]	train-map:0.94616	valid-map:0.94490
[200]	train-map:0.94749	valid-map:0.94530
[300]	train-map:0.94847	valid-map:0.94551
[400]	train-map:0.94928	valid-map:0.94555
[500]	train-map:0.94992	valid-map:0.94558
[600]	train-map:0.95052	valid-map:0.94560
[700]	train-map:0.95103	valid-map:0.94559
[800]	train-map:0.95160	valid-map:0.94555
[900]	train-map:0.95211	valid-map:0.94550
[999]	train-map:0.95265	valid-map:0.94552


2it [01:53, 56.43s/it]

...

3it [02:47, 55.20s/it]

[0]	train-map:0.94315	valid-map:0.94365
[100]	train-map:0.94602	valid-map:0.94581
[200]	train-map:0.94737	valid-map:0.94620
[300]	train-map:0.94820	valid-map:0.94623
[400]	train-map:0.94897	valid-map:0.94631
[500]	train-map:0.94963	valid-map:0.94640
[600]	train-map:0.95029	valid-map:0.94639
[700]	train-map:0.95082	valid-map:0.94643
[800]	train-map:0.95140	valid-map:0.94634
[900]	train-map:0.95192	valid-map:0.94634
[999]	train-map:0.95242	valid-map:0.94634


4it [03:41, 54.51s/it]

[0]	train-map:0.94337	valid-map:0.94283
[100]	train-map:0.94619	valid-map:0.94493
[200]	train-map:0.94756	valid-map:0.94524
[300]	train-map:0.94846	valid-map:0.94540
[400]	train-map:0.94926	valid-map:0.94551
[500]	train-map:0.94986	valid-map:0.94550
[600]	train-map:0.95054	valid-map:0.94554
[700]	train-map:0.95108	valid-map:0.94556
[800]	train-map:0.95159	valid-map:0.94555
[900]	train-map:0.95213	valid-map:0.94552
[999]	train-map:0.95273	valid-map:0.94550


5it [04:34, 54.92s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.1/XGB_fold2_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.1/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.1/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.1/XGB_fold1_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.1/XGB_fold4_orders.xgb']

## Предсказание

In [ ]:
model_version = 1.1
ver_folder = f'ver_{model_version}'
ver_prev = 'ver_1'

typee = 'clicks'
# typee = 'carts'
# typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_1.1/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_prev}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 10/10 [00:03<00:00,  2.86it/s]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020850, 8)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:30<00:00,  6.13s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
shutil.copy2(f'lv_predictions_{typee}.parquet', f'{main_path}/lv/{ver_folder}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_1.1/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_1.1'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')


In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.95it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.3362285716018827, type
carts     0.235359
clicks    0.236803
orders    0.403234
dtype: float64)


👁️**Model score : 0.336228**👁️

# Ver_1.2

Те же кандидаты (с версии 1) но именно предсказание другое, так как сохраненные настройки модели другие ( обучение с андерсемплингом)

## Обучение

In [ ]:
ver = '1.2'
ver_folder = f'ver_{ver}'
ver_prev = 'ver_1'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_prev}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 10/10 [00:31<00:00,  3.18s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

In [ ]:
#orders
tr_candidates['target'].value_counts()

0    77144955
1      234095
Name: target, dtype: int64

In [ ]:
#carts
tr_candidates['target'].value_counts()

0    77076368
1      302682
Name: target, dtype: int64

In [ ]:
#clicks
tr_candidates['target'].value_counts()

0    75316137
1     2062913
Name: target, dtype: int64

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.027390053209978095

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0039270402570084776

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0030344822937546597

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.3)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.05)  # Указываем желаемое соотношение классов

# # orders
under_sampler = RandomUnderSampler(sampling_strategy= 0.05)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    # Применяем under-sampling к данным
    # X_train, y_train = under_sampler.fit_resample(X_train, y_train)
    # X_valid, y_valid = under_sampler.fit_resample(X_valid, y_valid)

    # После under-sampling, получаем список с количеством объектов в каждой групп
    # X_for = tr_candidates.loc[train_idx, tr_candidates.columns[: -1]]
    # y_for = tr_candidates.loc[train_idx, 'target']
    # X_for, y_for = under_sampler.fit_resample(X_for, y_for)
    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    # dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) )
    # dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.97244	valid-map:0.97264
[100]	train-map:0.97488	valid-map:0.97403
[200]	train-map:0.97602	valid-map:0.97417
[300]	train-map:0.97690	valid-map:0.97427
[400]	train-map:0.97762	valid-map:0.97419
[500]	train-map:0.97828	valid-map:0.97417
[600]	train-map:0.97888	valid-map:0.97417
[700]	train-map:0.97945	valid-map:0.97414
[800]	train-map:0.98005	valid-map:0.97405
[900]	train-map:0.98056	valid-map:0.97406
[999]	train-map:0.98100	valid-map:0.97409


1it [00:22, 22.95s/it]

[0]	train-map:0.97253	valid-map:0.97232
[100]	train-map:0.97500	valid-map:0.97381
[200]	train-map:0.97616	valid-map:0.97400
[300]	train-map:0.97701	valid-map:0.97399
[400]	train-map:0.97777	valid-map:0.97403
[500]	train-map:0.97846	valid-map:0.97396
[600]	train-map:0.97907	valid-map:0.97396
[700]	train-map:0.97968	valid-map:0.97397
[800]	train-map:0.98022	valid-map:0.97391
[900]	train-map:0.98074	valid-map:0.97390
[999]	train-map:0.98123	valid-map:0.97385


2it [00:41, 20.52s/it]

...

3it [01:00, 19.76s/it]

[0]	train-map:0.97243	valid-map:0.97256
[100]	train-map:0.97495	valid-map:0.97370
[200]	train-map:0.97611	valid-map:0.97391
[300]	train-map:0.97704	valid-map:0.97395
[400]	train-map:0.97777	valid-map:0.97402
[500]	train-map:0.97843	valid-map:0.97408
[600]	train-map:0.97905	valid-map:0.97412
[700]	train-map:0.97963	valid-map:0.97407
[800]	train-map:0.98015	valid-map:0.97403
[900]	train-map:0.98067	valid-map:0.97398
[999]	train-map:0.98113	valid-map:0.97389


4it [01:19, 19.38s/it]

[0]	train-map:0.97264	valid-map:0.97223
[100]	train-map:0.97491	valid-map:0.97374
[200]	train-map:0.97605	valid-map:0.97403
[300]	train-map:0.97689	valid-map:0.97405
[400]	train-map:0.97757	valid-map:0.97414
[500]	train-map:0.97827	valid-map:0.97406
[600]	train-map:0.97885	valid-map:0.97403
[700]	train-map:0.97944	valid-map:0.97404
[800]	train-map:0.97996	valid-map:0.97394
[900]	train-map:0.98050	valid-map:0.97396
[999]	train-map:0.98093	valid-map:0.97392


5it [01:38, 19.63s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.2/XGB_fold4_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.2/XGB_fold2_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.2/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.2/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.2/XGB_fold1_orders.xgb']

## Предсказание

In [ ]:
model_version = 1.2
ver_folder = f'ver_{model_version}'
ver_prev = 'ver_1'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_1.2/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_prev}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 10/10 [00:03<00:00,  3.08it/s]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020850, 8)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:25<00:00,  5.10s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}'
os.makedirs(folder_path, exist_ok=True)

shutil.copy2(f'lv_predictions_{typee}.parquet', f'{folder_path}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_1.2/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_1.2'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.20it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.3373859732792238, type
carts     0.235964
clicks    0.233198
orders    0.405462
dtype: float64)


👁️**Model score : 0.337385**👁️

# Ver_1.3

Те же кандидаты (с версии 1) но именно предсказание другое, так как сохраненные настройки модели другие ( обучение с андерсемплингом)

## Обучение

In [ ]:
ver = '1.3'
ver_folder = f'ver_{ver}'
ver_prev = 'ver_1'
# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

random_state = 42

slice_value = 1

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'{main_path}/tr/{ver_prev}/tr_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
tr_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    tr_candidates = pd.concat([tr_candidates, df_], ignore_index=True)

100%|██████████| 10/10 [00:22<00:00,  2.23s/it]


In [ ]:
del df_

In [ ]:
tr_candidates.session = tr_candidates.session.astype(np.int32)
tr_candidates.aid = tr_candidates.aid.astype(np.int32)

In [ ]:
# Проверка на группы
len(tr_candidates) / 50 == tr_candidates.session.nunique()

True

In [ ]:
#orders
tr_candidates['target'].value_counts()

0    77144955
1      234095
Name: target, dtype: int64

In [ ]:
#carts
tr_candidates['target'].value_counts()

0    77076368
1      302682
Name: target, dtype: int64

In [ ]:
#clicks
tr_candidates['target'].value_counts()

0    75316137
1     2062913
Name: target, dtype: int64

**Соотношение классов**

In [ ]:
#Клики
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.027390053209978095

In [ ]:
#Картс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0039270402570084776

In [ ]:
#Ордерс
tr_candidates['target'].value_counts()[1] / tr_candidates['target'].value_counts()[0]

0.0030344822937546597

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# clicks
# under_sampler = RandomUnderSampler(sampling_strategy=0.4)  # Указываем желаемое соотношение классов

# carts
# under_sampler = RandomUnderSampler(sampling_strategy= 0.2)  # Указываем желаемое соотношение классов

# # orders
under_sampler = RandomUnderSampler(sampling_strategy= 0.2)  # Указываем желаемое соотношение классов


In [ ]:
FEATURES = tr_candidates.columns[: -1]
X = tr_candidates.loc[:, FEATURES]
y = tr_candidates.loc[:, 'target']
X_train, y_train = under_sampler.fit_resample(X, y)
tr_candidates = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [ ]:
folder_path = 'model'
os.makedirs(folder_path, exist_ok=True)

tr_candidates = tr_candidates.sort_values('session')
tr_candidates = tr_candidates.reset_index(drop = True)

skf = GroupKFold(n_splits=5)
FEATURES = tr_candidates.columns[: -1]

for fold,(train_idx, valid_idx) in tqdm(enumerate(skf.split(tr_candidates, tr_candidates['target'], groups=tr_candidates['session'] ))):

    X_train = tr_candidates.loc[train_idx, FEATURES]
    y_train = tr_candidates.loc[train_idx, 'target']
    X_valid = tr_candidates.loc[valid_idx, FEATURES]
    y_valid = tr_candidates.loc[valid_idx, 'target']

    # Применяем under-sampling к данным
    # X_train, y_train = under_sampler.fit_resample(X_train, y_train)
    # X_valid, y_valid = under_sampler.fit_resample(X_valid, y_valid)

    # После under-sampling, получаем список с количеством объектов в каждой групп
    # X_for = tr_candidates.loc[train_idx, tr_candidates.columns[: -1]]
    # y_for = tr_candidates.loc[train_idx, 'target']
    # X_for, y_for = under_sampler.fit_resample(X_for, y_for)
    group_sizes_tr = X_train.groupby('session')[['aid']].count()
    group_sizes_v = X_valid.groupby('session')[['aid']].count()

    X_train = X_train.iloc[:, 2:]
    X_valid = X_valid.iloc[:, 2:]

    dtrain = xgb.DMatrix(X_train, y_train, group=group_sizes_tr.aid.tolist() )
    dvalid = xgb.DMatrix(X_valid, y_valid, group=group_sizes_v.aid.tolist() )

    # dtrain = xgb.DMatrix(X_train, y_train, group=[50] * (len(train_idx)//50) )
    # dvalid = xgb.DMatrix(X_valid, y_valid, group=[50] * (len(valid_idx)//50) )

    xgb_parms = {'objective':'rank:pairwise',
                 'tree_method':'gpu_hist'
                #  'learning_rate' : 0.6,
                #  'max_depth' : 4
                 }
    model = xgb.train(xgb_parms,
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'model/XGB_fold{fold}_{typee}.xgb')

0it [00:00, ?it/s]

[0]	train-map:0.98468	valid-map:0.98433
[100]	train-map:0.98687	valid-map:0.98501
[200]	train-map:0.98802	valid-map:0.98504
[300]	train-map:0.98900	valid-map:0.98507
[400]	train-map:0.98982	valid-map:0.98494
[500]	train-map:0.99052	valid-map:0.98489
[600]	train-map:0.99125	valid-map:0.98484
[700]	train-map:0.99184	valid-map:0.98485
[800]	train-map:0.99237	valid-map:0.98485
[900]	train-map:0.99292	valid-map:0.98471
[999]	train-map:0.99334	valid-map:0.98463


1it [00:12, 12.30s/it]

[0]	train-map:0.98456	valid-map:0.98448
[100]	train-map:0.98678	valid-map:0.98552
[200]	train-map:0.98800	valid-map:0.98543
[300]	train-map:0.98892	valid-map:0.98531
[400]	train-map:0.98973	valid-map:0.98531
[500]	train-map:0.99043	valid-map:0.98523
[600]	train-map:0.99104	valid-map:0.98520
[700]	train-map:0.99162	valid-map:0.98513
[800]	train-map:0.99215	valid-map:0.98506
[900]	train-map:0.99266	valid-map:0.98512
[999]	train-map:0.99310	valid-map:0.98514


2it [00:22, 10.84s/it]

...

3it [00:31, 10.36s/it]

[0]	train-map:0.98469	valid-map:0.98451
[100]	train-map:0.98684	valid-map:0.98510
[200]	train-map:0.98801	valid-map:0.98509
[300]	train-map:0.98901	valid-map:0.98521
[400]	train-map:0.98984	valid-map:0.98512
[500]	train-map:0.99050	valid-map:0.98516
[600]	train-map:0.99115	valid-map:0.98508
[700]	train-map:0.99173	valid-map:0.98498
[800]	train-map:0.99225	valid-map:0.98489
[900]	train-map:0.99275	valid-map:0.98480
[999]	train-map:0.99312	valid-map:0.98480


4it [00:41, 10.13s/it]

[0]	train-map:0.98461	valid-map:0.98444
[100]	train-map:0.98681	valid-map:0.98533
[200]	train-map:0.98797	valid-map:0.98536
[300]	train-map:0.98891	valid-map:0.98536
[400]	train-map:0.98970	valid-map:0.98532
[500]	train-map:0.99038	valid-map:0.98529
[600]	train-map:0.99109	valid-map:0.98521
[700]	train-map:0.99170	valid-map:0.98512
[800]	train-map:0.99219	valid-map:0.98505
[900]	train-map:0.99265	valid-map:0.98506
[999]	train-map:0.99306	valid-map:0.98504


5it [00:51, 10.32s/it]


In [ ]:
folder_path = f'{main_path}/models/{ver_folder}/'
os.makedirs(folder_path, exist_ok=True)

file_list = glob.glob(f'/content/model/*{typee}.xgb')
[shutil.copy(file, f'{main_path}/models/{ver_folder}/') for file in file_list]

['/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.3/XGB_fold4_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.3/XGB_fold2_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.3/XGB_fold3_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.3/XGB_fold0_orders.xgb',
 '/content/drive/Othercomputers/Mac/Pet_project/models/ver_1.3/XGB_fold1_orders.xgb']

## Предсказание

In [ ]:
model_version = 1.3
ver_folder = f'ver_{model_version}'
ver_prev = 'ver_1'

# typee = 'clicks'
# typee = 'carts'
typee = 'orders'

In [ ]:
# импорт модели
shutil.copytree(f'{main_path}/models/{ver_folder}/', f'/content/{ver_folder}/')

'/content/ver_1.3/'

In [ ]:
# импорт кандидатов выбранного типа
shutil.copytree(f'{main_path}/lv/{ver_prev}/test_candidates_{typee}/', f'/content/test_candidates_{typee}/' )

'/content/lv_candidates_orders/'

In [ ]:
# Список файлов Parquet
file_list = glob.glob(f'test_candidates_{typee}/*')

# Создание пустого DataFrame для сбора данных
test_candidates = pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df_ = pd.read_parquet(file)

    # Объединение временного DataFrame с основным датасетом
    test_candidates = pd.concat([test_candidates, df_], ignore_index=True)

test_candidates = test_candidates.sort_values('session')
test_candidates = test_candidates.reset_index(drop = True)

100%|██████████| 10/10 [00:03<00:00,  3.08it/s]


In [ ]:
FEATURES = test_candidates.columns[2:]

In [ ]:
test_candidates.shape

(30020850, 8)

In [ ]:
preds = np.zeros(len(test_candidates))
for fold in tqdm(range(5)):
    model = xgb.Booster()
    model.load_model(f'{ver_folder}/XGB_fold{fold}_{typee}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})
    dtest = xgb.DMatrix(data=test_candidates[FEATURES])
    preds += model.predict(dtest)/5

100%|██████████| 5/5 [00:25<00:00,  5.11s/it]


In [ ]:
lv_predictions = test_candidates[['session','aid']].copy()
lv_predictions['pred'] = preds

In [ ]:
lv_predictions = lv_predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
lv_predictions['n'] = lv_predictions.groupby('session').aid.cumcount().astype('int8')
lv_predictions = lv_predictions.loc[lv_predictions.n<20]

In [ ]:
lv_predictions = lv_predictions.groupby('session').aid.apply(list)
lv_predictions = lv_predictions.to_frame().reset_index()

In [ ]:
lv_predictions.rename(columns = {'aid' : 'labels'}, inplace = True)

In [ ]:
lv_predictions.to_parquet(f'lv_predictions_{typee}.parquet')

In [ ]:
folder_path = f'{main_path}/lv/{ver_folder}'
os.makedirs(folder_path, exist_ok=True)

shutil.copy2(f'lv_predictions_{typee}.parquet', f'{folder_path}/lv_predictions_{typee}.parquet' )

'/content/drive/Othercomputers/Mac/Pet_project/lv/ver_1.3/lv_predictions_orders.parquet'

## Оценка метрики

In [ ]:
id2type_name = 'id2type.pkl'
version = 'ver_1.3'

In [ ]:
valid = pd.read_parquet(f'{main_path}/s/test_labels_loc.parquet')

In [ ]:
with open(f'{main_path}/pkl/{id2type_name}', 'rb') as file:
     id2type= pickle.load(file)

In [ ]:
# # Список файлов Parquet
file_list = glob.glob(f'{main_path}/lv/{version}/lv_predictions*.parquet')

# Создание пустого DataFrame для сбора данных
lv_predictions= pd.DataFrame()

# Цикл для чтения и объединения файлов
for file in tqdm(file_list):
    # Чтение файла Parquet во временный DataFrame
    df = pd.read_parquet(file)

    if 'carts' in file:
      df['type'] = 'carts'
    elif 'clicks' in file:
      df['type'] = 'clicks'
    else:
      df['type'] = 'orders'

    # Объединение временного DataFrame с основным датасетом
    lv_predictions = pd.concat([lv_predictions, df], ignore_index=True)

# lv_predictions.type = lv_predictions.type.map(lambda idx: id2type[idx])

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.06it/s]


In [ ]:
print('Model score :', metric_eval(lv_predictions, valid, id2type))

Model score : (0.33460697027116865, type
carts     0.233589
clicks    0.240806
orders    0.400750
dtype: float64)


👁️**Model score : 0.334606**👁️